# Aggregation Group By

In [1]:
import pandas as pd
import seaborn as sns

df = sns.load_dataset('titanic')

In [2]:
df.columns

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town',
       'alive', 'alone'],
      dtype='object')

In [3]:
df['fare'].agg(['sum', 'mean'])

sum     28693.949300
mean       32.204208
Name: fare, dtype: float64

In [4]:
df.agg({
    'fare': ['sum', 'mean'],
    'sex': ['count']
})

,fare,sex
count,NaN,891.0
mean,32.204208,NaN
sum,28693.949300,NaN


In [5]:
df.agg(
    fare_sum=('fare', 'sum'),
    fare_mean=('fare', 'mean'),
    sex_count=('sex', 'count')
)

,fare,sex
fare_sum,28693.949300,NaN
fare_mean,32.204208,NaN
sex_count,NaN,891.0


In [6]:
df.groupby(['embark_town']).agg({
    'fare': ['sum', 'mean', 'median', 'min', 'max', 'std', 'var', 'mad', 'prod']
}).round(2)

fare                      ...                                      
                  sum   mean median   min  ...    std      var    mad           prod
embark_town                                ...                                      
Cherbourg    10072.30  59.95  29.70  4.01  ...  83.91  7041.39  53.02  6.193716e+250
Queenstown    1022.25  13.28   7.75  6.75  ...  14.19   201.30   7.87   6.458671e+78
Southampton  17439.40  27.08  13.00  0.00  ...  35.89  1287.95  21.30   0.000000e+00

[3 rows x 9 columns]

In [7]:
df.groupby(['embark_town']).agg({
    'fare': ['describe']
})

fare                        ...                            
            describe                        ...                            
               count       mean        std  ...    50%        75%       max
embark_town                                 ...                            
Cherbourg      168.0  59.954144  83.912994  ...  29.70  78.500025  512.3292
Queenstown      77.0  13.276030  14.188047  ...   7.75  15.500000   90.0000
Southampton    644.0  27.079812  35.887993  ...  13.00  27.900000  263.0000

[3 rows x 8 columns]

In [8]:
df.groupby(['deck']).agg({
    'embark_town': ['count', 'nunique', 'size']
})

embark_town             
           count nunique size
deck                         
A             15       2   15
B             45       2   47
C             59       3   59
D             33       2   33
E             32       3   32
F             13       3   13
G              4       1    4

In [9]:
df.sort_values(by=['fare'], ascending=False) \
  .groupby(['embark_town']) \
  .agg({
      'fare': ['first', 'last'] # max, min
  })

fare        
                first    last
embark_town                  
Cherbourg    512.3292  4.0125
Queenstown    90.0000  6.7500
Southampton  263.0000  0.0000

In [10]:
df.sort_values(by=['fare'], ascending=False) \
  .groupby(['embark_town']) \
  .agg({
      'fare': ['max', 'min'] # max, min
  })

fare        
                  max     min
embark_town                  
Cherbourg    512.3292  4.0125
Queenstown    90.0000  6.7500
Southampton  263.0000  0.0000

In [12]:
df.sort_values(by=['fare'], ascending=False) \
  .groupby(['embark_town']) \
  .agg({
      'fare': ['idxmax', 'idxmin']
  })

fare       
            idxmax idxmin
embark_town              
Cherbourg      258    378
Queenstown     412    654
Southampton     88    466

In [13]:
df.loc[[258, 378]]

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
258,1,1,female,35.0,0,0,512.3292,C,First,woman,False,NaN,Cherbourg,yes,True
378,0,3,male,20.0,0,0,4.0125,C,Third,man,True,NaN,Cherbourg,no,True


In [14]:
# shortcut
df.loc[df.groupby('class')['fare'].idxmax()]

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
258,1,1,female,35.0,0,0,512.3292,C,First,woman,False,NaN,Cherbourg,yes,True
72,0,2,male,21.0,0,0,73.5000,S,Second,man,True,NaN,Southampton,no,True
159,0,3,male,NaN,8,2,69.5500,S,Third,man,True,NaN,Southampton,no,False


In [15]:
from scipy.stats import skew, mode

df.groupby(['embark_town']).agg({
    'fare': [skew, mode, pd.Series.mode]
})

fare                          
                 skew              mode    mode
embark_town                                    
Cherbourg    3.305112  ([7.2292], [15])  7.2292
Queenstown   4.265111    ([7.75], [30])  7.7500
Southampton  3.640276    ([8.05], [43])  8.0500

In [16]:
df.groupby(['class']).agg({
    'deck': ['nunique', mode, set]
})

deck                                   
       nunique         mode                   set
class                                            
First        5  ([C], [59])  {nan, C, D, E, A, B}
Second       3   ([F], [8])        {nan, E, D, F}
Third        3   ([F], [5])        {nan, G, E, F}

In [17]:
# custom agg functions

from functools import partial

q_25 = partial(pd.Series.quantile, q=0.25)
q_25.__name__ = '25%'

# ou

def percentile_25(x):
  return x.quantile(.25)

# ou

lambda_25 = lambda x: x.quantile(.25)
lambda_25.__name__ = 'lambda_25%'

In [18]:
df.groupby(['embark_town']).agg({
    'fare': [q_25, percentile_25, lambda_25, lambda x: x.quantile(.25)]
})

fare                                    
                  25% percentile_25 lambda_25% <lambda_0>
embark_town                                              
Cherbourg    13.69795      13.69795   13.69795   13.69795
Queenstown    7.75000       7.75000    7.75000    7.75000
Southampton   8.05000       8.05000    8.05000    8.05000

In [19]:
def count_nulls(s):
  return s.size - s.count()

df.groupby(['deck']).agg({
    'embark_town': ['count', 'nunique', 'size', count_nulls, set]
})

embark_town                                                               
           count nunique size count_nulls                                   set
deck                                                                           
A             15       2   15           0              {Southampton, Cherbourg}
B             45       2   47           2         {nan, Southampton, Cherbourg}
C             59       3   59           0  {Southampton, Cherbourg, Queenstown}
D             33       2   33           0              {Southampton, Cherbourg}
E             32       3   32           0  {Southampton, Cherbourg, Queenstown}
F             13       3   13           0  {Southampton, Cherbourg, Queenstown}
G              4       1    4           0                         {Southampton}

In [24]:
def top_10_sum(x):
  return x.nlargest(10).sum()

def bottom_10_sum(x):
  return x.nsmallest(10).sum()

df.groupby(['class']).agg({
    'fare': [top_10_sum, bottom_10_sum]
})

fare              
       top_10_sum bottom_10_sum
class                          
First   3361.2584      108.3709
Second   622.2376       42.0000
Third    656.3374       36.1291

If you have a scenario where you want to run multiple aggregations across columns, then you may want to use the groupby combined with apply as described in this stack overflow answer.
https://stackoverflow.com/questions/14529838/apply-multiple-functions-to-multiple-groupby-columns/47103408#47103408

In [25]:
def my_summary(x):
  result = {
      'fare_sum': x['fare'].sum(),
      'fare_mean': x['fare'].mean(),
      'fare_range': x['fare'].max() - x['fare'].min()
  }
  return pd.Series(result).round(0)

df.groupby(['class']).apply(my_summary)

,fare_sum,fare_mean,fare_range
class,,,
First,18177.0,84.0,512.0
Second,3802.0,21.0,74.0
Third,6715.0,14.0,70.0


Using apply with groupy gives maximum flexibility over all aspects of the results. However, there is a downside. The apply function is slow so this approach should be used sparingly.

In [27]:
df.groupby(['embark_town', 'class']).agg({
    'fare': ['sum']
}).assign(
  pct_total=lambda x: x / x.sum()
)

fare pct_total
                          sum          
embark_town class                      
Cherbourg   First   8901.0750  0.311947
            Second   431.0917  0.015108
            Third    740.1295  0.025939
Queenstown  First    180.0000  0.006308
            Second    37.0500  0.001298
            Third    805.2043  0.028219
Southampton First   8936.3375  0.313183
            Second  3333.7000  0.116833
            Third   5169.3613  0.181165

In [39]:
df.groupby(['embark_town', 'class']) \
  .agg({'fare': 'sum'}) \
  .groupby(level=0) \
  .cumsum() # soma cumulativa

fare
embark_town class             
Cherbourg   First    8901.0750
            Second   9332.1667
            Third   10072.2962
Queenstown  First     180.0000
            Second    217.0500
            Third    1022.2543
Southampton First    8936.3375
            Second  12270.0375
            Third   17439.3988

In [40]:
df.groupby(['embark_town']).agg({'fare': 'sum'})

,fare
embark_town,
Cherbourg,10072.2962
Queenstown,1022.2543
Southampton,17439.3988


# Aggregation Crosstab

In [30]:
pd.crosstab(df['embark_town'], 
            df['class'], 
            values=df['fare'], 
            aggfunc='sum', 
            normalize=True)

class,First,Second,Third
embark_town,,,
Cherbourg,0.311947,0.015108,0.025939
Queenstown,0.006308,0.001298,0.028219
Southampton,0.313183,0.116833,0.181165


# Aggregation Pivot Table

In [34]:
pd.pivot_table(data=df,
               index=['embark_town'],
               columns=['class'],
               aggfunc={'fare': ['mean', 'sum']})

fare                                                       
                   mean                              sum                      
class             First     Second      Third      First     Second      Third
embark_town                                                                   
Cherbourg    104.718529  25.358335  11.214083  8901.0750   431.0917   740.1295
Queenstown    90.000000  12.350000  11.183393   180.0000    37.0500   805.2043
Southampton   70.364862  20.327439  14.644083  8936.3375  3333.7000  5169.3613

# Aggregation Grouper

# Style